In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from db_manager import SQLiteWrapper
from functions_notion import *

In [2]:
sqlite_path = "KoboReader.sqlite"

In [3]:
pd.options.display.max_columns = None

# Carga de BBDD

In [4]:
db_path = "KoboReader.sqlite"  # Ruta a tu archivo SQLite
db = SQLiteWrapper(db_path)
db.connect()  # Establecer conexión
anotaciones_df = db.get_annotations()
libros_ereader_df = db.get_books()
# db.close()

# Dropbox

In [ ]:
from functions_dropbox import get_epub_metadata_from_dropbox 
from config import APP_KEY, APP_SECRET, TOKEN_FILE

In [6]:
epub_metadata = get_epub_metadata_from_dropbox(APP_KEY, APP_SECRET, TOKEN_FILE, folder_path='/Aplicaciones/Rakuten Kobo')
epub_metadata.to_pickle("epub_metadata.pkl")

Fallo en The_Rational_Male.epub
'title'
Fallo en Left_Swipes_Love_A_Millennials_Guide_to_Hookups_Dating_and_.epub
'title'
Error al procesar el archivo toc.ncx: No se encontró ningún archivo toc.ncx en el EPUB.
Fallo en How to Win at Chess_ The Ultimate Guide for Beginners and -- Levy Rozman -- 2023 -- Clarkson Potter_Ten Speed -- 1984862081 -- cb65fc4e7be857658c8b8c3e11a93cae -- Anna’s Archive.epub
'title'
Fallo en SUMMARYThe_Six_Pillars_of_SelfEsteem_by_Nathaniel_Branden.epub
'title'
Fallo en Hill - Stone - La Actitud Mental Positiva -- Edited by Fabricio Bossio, ELEVEN - Biblioteca del Nuevo -- b5290d34663758be225b9aaf114623a2 -- Anna’s Archive.epub
'title'
Fallo en The_Slight_Edge_Turning_Simple_Disciplines_into_Massive_Success_.epub
'title'
Fallo en Grit_Angela_Duckworth.epub
'title'
Fallo en Models -- Mark Manson -- Mark Manson -- 62f7c7c925b50cdbfdbafc78bb65ec1e -- Anna’s Archive.epub
'title'
Fallo en How_to_Stop_Worrying_and_Start_Living.epub
'title'
Fallo en Google Cloud Certif

In [ ]:
# epub_metadata = pd.read_pickle("epub_metadata.pkl")

In [8]:
libros_anotaciones_df = anotaciones_df\
    [['Autor', 'Título','Fecha de creación']]\
    .assign(num_anotaciones = 1)\
    .groupby(['Autor', 'Título'])\
    .agg({"Fecha de creación": ["min", "max"], "num_anotaciones": "sum"})\
    .reset_index()\
    .set_axis(['autor', 'titulo', 'fecha_primera_nota', 'fecha_ultima_nota', 'num_anotaciones'], axis = 1)

def clean_generos(generos):
    if len(generos) == 1 and isinstance(generos[0], str):
        return generos[0].split(", ")
    return generos

libros_dropbox = epub_metadata\
    .rename(columns = {"title": "titulo",
                       "author": "autor", "subjects": "generos", "pages": "paginas",
                       "publication_date": "fecha_publicacion"})\
    .assign(idioma = lambda x: x.language.str.extract("(es|en)").fillna("en"))\
    .assign(idioma = lambda x: x.idioma.map({"es": "Español", "en": "Inglés"}))\
    .assign(generos = lambda x: x.generos.apply(clean_generos))\
    [['autor', 'titulo', 'generos', 'paginas', 'fecha_publicacion']]

libros_df = libros_ereader_df\
    .merge(libros_anotaciones_df, how = "left")\
    .merge(libros_dropbox, on = ["autor", "titulo"], how = "left")\
    .loc[lambda x: x.autor.notna()]\
    .assign(num_anotaciones = lambda x: x.num_anotaciones.fillna(0))

# Exportacion a Notion

In [ ]:
from notion_client import Client
from config import NOTION_API_TOKEN, NOTION_BOOKS_DATABASE_ID, NOTION_ANNOTATIONS_DATABASE_ID

# Configuración inicial
notion = Client(auth=NOTION_API_TOKEN)

In [11]:
create_books(libros_df, notion, NOTION_BOOKS_DATABASE_ID)

Fallo en ('La playa de los ahogados', 'Domingo Villar')
Request to Notion API has timed out
Traceback (most recent call last):
  File "C:\Users\jaguirrepeman\AppData\Roaming\Python\Python312\site-packages\httpx\_transports\default.py", line 101, in map_httpcore_exceptions
    yield
  File "C:\Users\jaguirrepeman\AppData\Roaming\Python\Python312\site-packages\httpx\_transports\default.py", line 250, in handle_request
    resp = self._pool.handle_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jaguirrepeman\AppData\Roaming\Python\Python312\site-packages\httpcore\_sync\connection_pool.py", line 256, in handle_request
    raise exc from None
  File "C:\Users\jaguirrepeman\AppData\Roaming\Python\Python312\site-packages\httpcore\_sync\connection_pool.py", line 236, in handle_request
    response = connection.handle_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jaguirrepeman\AppData\Roaming\Python\Python312\site-packages\httpcore\_sync\connection.

In [12]:
create_annotations(anotaciones_df, notion, NOTION_ANNOTATIONS_DATABASE_ID, NOTION_BOOKS_DATABASE_ID, 
                   delete_existing=True)

In [13]:
create_book_pages(anotaciones_df, notion, NOTION_BOOKS_DATABASE_ID, clear_content = True)

Contenido existente eliminado para el libro '12 reglas para vivir'.
Contenido existente eliminado para el libro 'A puerta cerrada'.
Contenido existente eliminado para el libro 'Acontece que no es poco'.
Contenido existente eliminado para el libro 'Amanecer'.
Contenido existente eliminado para el libro 'Attached: Are you Anxious, Avoidant or Secure? How the science of adult attachment can help you find – and keep – love'.
Contenido existente eliminado para el libro 'Breaking the Habit of Being Yourself'.
Contenido existente eliminado para el libro 'Como hacer que te pasen cosas buenas'.
Contenido existente eliminado para el libro 'Cómo invertir en Bolsa a largo plazo partiendo de cero'.
Contenido existente eliminado para el libro 'Cómo invertir en fondos de inversión con sentido común'.
Contenido existente eliminado para el libro 'Cómo piensan los ricos'.
Contenido existente eliminado para el libro 'Educación financiera avanzada partiendo de cero'.
Contenido existente eliminado para el 

In [ ]:
anotaciones_df_12 = anotaciones_df\
    .loc[lambda x: x.Título == "12 reglas para vivir"]

In [ ]:
create_book_pages(anotaciones_df_12, notion, NOTION_BOOKS_DATABASE_ID, clear_content = True)

Contenido existente eliminado para el libro '12 reglas para vivir'.


In [ ]:
notion_books_df = get_books_from_notion(notion, NOTION_BOOKS_DATABASE_ID)

Error procesando el libro con ID 1a215076-86e6-800d-b5b2-de06d5d4b2fc: 'NoneType' object has no attribute 'get'
Error procesando el libro con ID 1a015076-86e6-819d-9c38-cb91484bf95f: 'NoneType' object has no attribute 'get'
Error procesando el libro con ID 1a015076-86e6-81c4-834a-e23f09a2729b: 'NoneType' object has no attribute 'get'
Error procesando el libro con ID 1a015076-86e6-81f2-a833-d51495abdeec: 'NoneType' object has no attribute 'get'
Error procesando el libro con ID 1a015076-86e6-8165-b760-d9fc6a9e8141: 'NoneType' object has no attribute 'get'
Error procesando el libro con ID 1a015076-86e6-810e-b142-da320ce4628e: 'NoneType' object has no attribute 'get'
Error procesando el libro con ID 1a015076-86e6-813c-91b9-dbf51c44aec6: 'NoneType' object has no attribute 'get'
Error procesando el libro con ID 1a015076-86e6-8181-ba1e-cf01de170e00: 'NoneType' object has no attribute 'get'
Error procesando el libro con ID 1a015076-86e6-8139-8af8-da1f356643d7: 'NoneType' object has no attribut

In [ ]:
notion_books_df.sort_values(["titulo", "autor"]).shape

(71, 10)

# Dropbox

# Exploratorio tablas interesantes

In [ ]:
QUERY_ITEMS = """
    SELECT 
        b.VolumeID, 
        b.Text, 
        b.Annotation, 
        b.DateCreated, 
        b.DateModified,
        b.ChapterProgress, 
        b.Type as Type,
        c.BookTitle, 
        c.Title as Chapter, 
        c.Attribution as Author, 
        b.BookmarkID
    FROM Bookmark b INNER JOIN content c
    ON b.ContentID = c.ContentID 
    GROUP BY b.DateCreated 
    ORDER BY b.ChapterProgress ASC, b.DateCreated ASC;
"""

QUERY_ITEMS2 = """
    SELECT 
        b.VolumeID, 
        b.Text, 
        b.Annotation, 
        b.DateCreated, 
        b.DateModified,
        b.ChapterProgress, 
        b.Type as Type,
        c.BookTitle, 
        c.Title as Chapter, 
        c.Attribution as Author, 
        b.BookmarkID
    FROM Bookmark b INNER JOIN content c
    ON b.VolumeID = c.ContentID 
    GROUP BY b.DateCreated 
    ORDER BY b.ChapterProgress ASC, b.DateCreated ASC;
"""


QUERY_BOOKS = """
    SELECT DISTINCT
        b.VolumeID,
        c.BookTitle, 
        c.Title,
        c.Attribution as Author,
        (SELECT COUNT(*) FROM Bookmark b2 WHERE b2.VolumeID = b.VolumeID) AS Items
    FROM
        Bookmark b INNER JOIN content c ON b.VolumeID = c.ContentID
    ORDER BY
        c.Title;
"""

In [ ]:
content_df.astype(str).apply(lambda x: x.str.contains("arte de amar"))\
    .sum().loc[lambda x: x>0]

In [ ]:
content_df\
    .loc[lambda x: x.BookTitle.notna()]\
    .loc[lambda x: x.BookTitle.str.contains("arte de amar")]

In [ ]:
anotaciones_clean_df = anotaciones_df\
    .loc[lambda x: x.Type.isin(["highlight", "note"])]\
    [['BookTitle', 'VolumeID', 'Chapter', 'ChapterProgress', 'Text', 'Annotation', 'Type', 'DateCreated']]

In [ ]:
anotaciones_clean_df.head(1)

In [ ]:
libros_df.drop(["BookTitle", "Items"], axis = 1)

In [ ]:
content_df = db.get_query_df("""select * from content""")
nunique_cols_content = db.get_query_df("select * from content").nunique().loc[lambda x: x<= 1].index
content_df = content_df.drop(nunique_cols_content, axis = 1)
bookmark_df = db.get_query_df("""select * from Bookmark""")
nunique_cols_bookmark = db.get_query_df("select * from Bookmark").nunique().loc[lambda x: x<= 1].index
bookmark_df = bookmark_df.drop(nunique_cols_bookmark, axis = 1)

In [ ]:
anotaciones_df = db.get_query_df(QUERY_ITEMS)
anotaciones_df2 = db.get_query_df(QUERY_ITEMS2)

In [ ]:
anotaciones_df.isna().sum().loc[lambda x: x>0]

In [ ]:
anotaciones_df2.isna().sum().loc[lambda x: x>0]

In [ ]:
anotaciones_df2.shape

In [ ]:
anotaciones_df.head(1)

In [ ]:
anotaciones_df2.head(1)

In [ ]:
libros_df = db.get_query_df(QUERY_BOOKS)

In [ ]:
libros_df.head().VolumeID.values[0]

In [ ]:
QUERY_CHAPTERS = """
    SELECT DISTINCT
        b.VolumeID,
        c.BookTitle, 
        c.Title,
        c.Attribution as Author
    FROM
        Bookmark b INNER JOIN content c ON b.ContentID = c.ContentID
    ORDER BY
        c.Title;
"""
capitulos_df = db.get_query_df(QUERY_CHAPTERS)

In [ ]:
bookmark_df.head(1)

In [ ]:
content_df.loc[lambda x: x.BookTitle.notna()].loc[lambda x: x.BookTitle.str.contains("12")]\
    .nunique().loc
    # .iloc[0]

In [ ]:
db.get_query_df("""select * from Bookmark
                 where VolumeID like '%12_regl%' and Text is not Null
                  """)#[['VolumeID', 'ContentID']].VolumeID.unique()

In [ ]:
## Database Schema Reference

**content table:**
- ContentID: 'file:///mnt/onboard/.kobo/dropbox/12_reglas_para_vivir_Jordan_Peterson.epub#(0)OEBPS/Text/cubierta.xhtml'
- BookID: 'file:///mnt/onboard/.kobo/dropbox/12_reglas_para_vivir_Jordan_Peterson.epub'

**Bookmark table:**
- VolumeID: 'file:///mnt/onboard/.kobo/dropbox/12_reglas_para_vivir_Jordan_Peterson.epub'
- ContentID: 'file:///mnt/onboard/.kobo/dropbox/12_reglas_para_vivir_Jordan_Peterson.epub#(7)OEBPS/Text/cap005.xhtml'

In [ ]:
capitulos_df

In [ ]:
anotaciones_df2.loc[lambda x: x.VolumeID.str.contains("12_reglas")].VolumeID.unique()

In [ ]:
anotaciones_df.loc[lambda x: x.VolumeID.str.contains("12_reglas")].VolumeID.unique()

In [ ]:
content_df.loc[lambda x: x.ContentID.str.contains("12_reglas")]

In [ ]:
bookmark_df

## Exploratorio de Todas las Tablas

In [ ]:
db_path = "KoboReader.sqlite"  # Ruta a tu archivo SQLite
db = SQLiteWrapper(db_path)
db.connect()  # Establecer conexión

In [ ]:
tables_info_df = db.get_tables_info() 

Tablas interesantes:
* content: contenido interesante
* Event
* Bookmark

Otras tablas:
* Shelf: Colecciones de libros. Por ejemplo, Histórica, Thriller, Filosofía...
* ShelfContent: Contenidos de las colecciones de Shelf
* WordList: Lista de palabras guardadas
* DropboxItem: Contenidos de dropbox
* user: Información del usuario. Revisar
* content_settings: configuración de lectura de varios libros
* Achievement: Premios de lectura

In [ ]:
tables_info_df.sort_values("num_filas")

In [ ]:
db.get_query_df("select * from content limit 10")

In [ ]:
db.get_query_df("select * from DbVersion")

In [ ]:
db.get_query_df("select * from volume_shortcovers limit 10 ")

In [ ]:
db.get_query_df("select * from user")

In [ ]:
db.get_query_df("select * from Tab")

In [ ]:
db.get_query_df("select * from KoboPlusAssetGroup")

In [ ]:
db.get_query_df("select * from GDriveItem")

In [ ]:
db.get_query_df("select * from Shelf")

In [ ]:
db.get_query_df("select * from AnalyticsEvents")

In [ ]:
db.get_query_df("select * from SubscriptionProducts")

In [ ]:
db.get_query_df("select * from content_settings limit 5")

In [ ]:
db.get_query_df("select * from Achievement")\
        .loc[lambda x: x.PercentComplete == 100].CompleteDescription.values

In [ ]:
db.get_query_df("select * from Rules limit 5")

In [ ]:
db.get_query_df("select * from ShelfContent limit 5")

In [ ]:
db.get_query_df("select * from WordList ")

In [ ]:
db.get_query_df("select * from Activity ")

In [ ]:
db.get_query_df("select * from DropboxItem").Json[0]

In [ ]:
db.get_query_df("select * from KoboPlusAssets where Language = 'es' ").Value.values

In [ ]:
db.get_query_df("select * from Event limit 10 ")

In [ ]:
db.get_query_df("select * from Bookmark limit 10 ")

In [ ]:
db.get_query_df("select * from volume_shortcovers limit 10 ")

In [ ]:
db.get_query_df("""select *
                from content limit 10
                """)

In [ ]:
db.get_query_df("""select BookID, BookTitle, count(*)
                from content 
                group by 1,2
                order by 3 desc""")

In [ ]:
tables_info_df.sort_values("num_filas")